In [1]:
import requests
import os
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from io import BytesIO

In [2]:
start = "https://www.wikipedia.org"
all_html= requests.get(start)
print("Status code:", all_html.status_code)


Status code: 200


In [3]:
content_html = BeautifulSoup(all_html.content, "html.parser")
link = []
for a_tag in content_html.find_all('a', href=True):
    link.append(urljoin(start, a_tag['href']))
    
print("Links found:", len(link))

Links found: 371


In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from PIL import Image
from io import BytesIO

images = []
folder_name = "downloaded_images"
os.makedirs(folder_name, exist_ok=True)
image_count = 0

for page_url in link:
    response = requests.get(page_url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        
        for img_tag in soup.find_all('img', src=True):
            img_src = img_tag['src']
            img_url = urljoin(page_url, img_src)
            
            try:
                img_response = requests.get(img_url, timeout=10)
                img = Image.open(BytesIO(img_response.content))
                width, height = img.size

                if width < 100 or height < 100:
                    print(f"Skipped small image: {img_url} ({width}x{height})")
                    continue

                file_name = os.path.join(folder_name, f"image_{image_count}.jpg")
                img.save(file_name)  # Saves directly via PIL
                images.append(img_url)
                print(f" Downloaded: image_{image_count}.jpg ({width}x{height})")
                image_count += 1

            except Exception as e:
                print(f"Failed to download or process {img_url}: {e}")

print(f"\n Total images downloaded: {image_count}")
